Plotly example
==============

see https://plot.ly/python/cars-exploration/ (https://github.com/jonmmease/plotly_ipywidget_notebooks/blob/master/notebooks/cars_exploration.ipynb)


make sure proper extensions are installed, as per
   https://github.com/plotly/plotly.py
   
Also, FigureWidgets cannot be exported to HTML straight from jupyter lab. (see https://community.plot.ly/t/export-figurewidgets-from-jupyter-notebook-to-html/15556/2)
Use the command line :
```
jupyter nbconvert --to html viz_tests/plotly_test.ipynb 
```

In [312]:
import pandas as pd
data = pd.read_json('../votes_pcs.json', orient='records')

# read MEPs details
with open('../meps_summary.json') as json_file:  
    meps_details = json.load(json_file)

In [314]:
import plotly.plotly as py
import plotly.graph_objs as go

# taken from https://en.wikipedia.org/wiki/Political_groups_of_the_European_Parliament
group_colors = {
    'PPE':'#3399FF',
    'S&D':'#FF0000',
    #'PES':'#FF0000', # old name of S&D
    'ECR':'#0054A5',
    'ALDE':'#FFD700',
    'GUE/NGL':'#990000',
    'Verts/ALE':'#009900',
    'EFDD':'#24B9B9',
    'ENF':'#2B3856',
    'NA':'#999999'
}
groups = data['group'].unique()
countries = data['country'].unique()

maxVotes = data['votes_count'].max()
minVotes = data['votes_count'].min()
total_votes = maxVotes # wrong, but that's the best we got now
markerSizes = { group:[ 1+8*(count-minVotes)/(maxVotes-minVotes) for count in data[data['group']==group]['votes_count'] ] for group in groups }

pca_minmax = { pca_name:[data[pca_name].min(), data[pca_name].max()] for pca_name in ['PCA0', 'PCA1', 'PCA2'] }

grpButtonActions = [
    {'label': 'All Parties',
     'method': 'restyle',
     'args': [{
             'marker.opacity' : 1
         }]
    }
]
for idx, groupName in enumerate(groups):
    grpButtonActions.append(
        {'label': groupName,
         'method': 'restyle',
         'args': [
             {
                 #'marker.size': [ #2d array : grouped per trace, then per point within trace 
                 #    [2*size for size in markerSizes[groupName]] if idx == idx2 else markerSizes[groupName] for idx2, groupName in enumerate(groups)
                 #],
                 # 'marker.line.width'  # useless bc of https://github.com/plotly/plotly.js/issues/3796
                 'marker.opacity' : [
                     ( 1 if idx == idx2 else 0.1) for idx2, groupName in enumerate(groups) 
                  ]
             },
             # pass array of trace idxs to apply only to scatter traces, not mesh
             #[idx2 for idx2, groupName in enumerate(groups)]
         ]
        })
    

countryButtonActions = [
    {'label': 'All Countries',
     'method': 'restyle',
     'args': [{
             'marker.color' : [ group_colors[groupName] for idx2, groupName in enumerate(groups) ],
             'marker.line.color' : 'black'
         }]
    }
]
for idx, country in enumerate(countries):
    countryButtonActions.append(
        {'label': country,
         'method': 'restyle',
         'args': [
             {
                 #'marker.size': [ #2d array : grouped per trace, then per point within trace 
                 #    [2*size for size in markerSizes[groupName]] if idx == idx2 else markerSizes[groupName] for idx2, groupName in enumerate(groups)
                 #],
                 'marker.color' : [
                     [( group_colors[groupName] if mep_country == country else 'rgba(0,0,0,0.1)') for mep_country in data[data['group']==groupName]['country']] for idx2, groupName in enumerate(groups) 
                  ],
                 #'marker.opacity' : [ # DOES NOT WORK
                 #    [( 1 if mep_country == country else 0.1) for mep_country in data[data['group']==groupName]['country']] for idx2, groupName in enumerate(groups) 
                 # ],
                 'marker.line.color' : [
                     [( 'black' if mep_country == country else 'transparent') for mep_country in data[data['group']==groupName]['country']] for idx2, groupName in enumerate(groups) 
                  ],
             },
             # pass array of trace idxs to apply only to scatter traces, not mesh
             #[idx2 for idx2, groupName in enumerate(groups)]
         ]
        })
    
    

updatemenus=list([
    {
        'buttons':grpButtonActions,
        'direction': 'down',
        #'pad': {'r': 10, 't': 10},
        'showactive': True,
        'x': 1.2,
        'xanchor': 'right',
        'y': .65,
        'yanchor': 'top' 
    },
    {
        'buttons':countryButtonActions,
        'direction': 'down',
        #'pad': {'r': 10, 't': 10},
        'showactive': True,
        'x': 1.2,
        'xanchor': 'right',
        'y': .57,
        'yanchor': 'top' 
    }
])    
    
# Some generic plotly layout
layout_comp = go.Layout(
    title='MEPs PCAs',
    hovermode='closest',
    hoverdistance=50,
#    width=500,
    height=700,
    scene = {
        'xaxis' : {
            'range': [pca_minmax['PCA0'][0], pca_minmax['PCA0'][1]],
            'autorange': False,
            'title':'PCA0',
            'ticklen':5,
            'zeroline':False,
            'gridwidth':2,
        },
        'yaxis' : {
            'range': [pca_minmax['PCA1'][0], pca_minmax['PCA1'][1]],
            'autorange': False,
            'title':'PCA1',
            'ticklen':5,
            'zeroline':False,
            'gridwidth':2,
        },
        'zaxis' : {
            'range': [pca_minmax['PCA2'][0], pca_minmax['PCA2'][1]],
            'autorange': False,
            'title':'PCA2',
            'ticklen':5,
            'zeroline':False,
            'gridwidth':2,
        }
    },
    updatemenus = updatemenus
)

traces = [
        {
            'x': data[data['group']==group]['PCA0'],
            'y': data[data['group']==group]['PCA1'],
            'z': data[data['group']==group]['PCA2'],
            'type': 'scatter3d',
            'name': group,
            'text': [ "<b>%s %s</b><br>%.2f %%<br>%s<br>%s<br>%s" % (
                meps_details[str(mep['mep_id'])]['surname'], 
                meps_details[str(mep['mep_id'])]['name'], 
                100*mep['votes_count']/total_votes,
                meps_details[str(mep['mep_id'])]['current_constituency'], 
                meps_details[str(mep['mep_id'])]['current_group'], 
                meps_details[str(mep['mep_id'])]['country']) 
                     for idx, mep in data[data['group']==group][['votes_count', 'mep_id']].iterrows() ],
            'hoverinfo': 'text',
            'mode': 'markers',
            'marker': {'size': markerSizes[group],
                       'opacity' : 1,
                       'color': group_colors[group],
                       'line': {
                           'width':1, 
                           'color' : 'black'},
                      }
        } for group in groups
    ]

# highlight a group with a 3d mesh (cf https://plot.ly/python/3d-mesh/)
'''
mesh3d = {  'x': data[data['group']=='EFDD']['PCA0'],
            'y': data[data['group']=='EFDD']['PCA1'],
            'z': data[data['group']=='EFDD']['PCA2'],
            'type': 'mesh3d',
            'alphahull':5,
                   'opacity':0.05,
                   'color':'#0000DD'} 
traces.append(mesh3d)
'''

# Create the plotly figure and plot it
fig_comp = go.FigureWidget(data=traces, layout=layout_comp)

traces = fig_comp.data 

# use widgets to have event handling
from ipywidgets import Output, VBox
#out = Output()

import webbrowser
#@out.capture(clear_output=True)
def handle_click(trace, points, state):
    if (len (points.point_inds) > 0):
        #print (points.point_inds[0])
        mep_id = data[data['group']==trace['name']]['mep_id'].values[points.point_inds[0]]
        webbrowser.open_new_tab(meps_details[str(mep_id)]['eu_homepage'])   

for trace in traces : trace.on_click(handle_click)

VBox([fig_comp, out])


    'data': [{'hoverinfo': 'text',
              'marker': {'color': '#3399FF',
…